In [1]:
#%% IMPORTAR LIBRERIAS
import os
import time
import tqdm
import argparse
import shutil
import zipfile

import imageio
from PIL import Image

import warnings
warnings.filterwarnings('ignore')

from moviepy.editor import VideoFileClip


In [25]:
original_wdir = False
gif_extension = '.webp'

sourcetype = 'webp' # ['webp', 'images', 'video']
iszip = False
verbose = True

infolder = 'gif_builder/' + 'input/' + sourcetype + '/'
outfolder = 'gif_builder/output/'

In [3]:
# cambiar directorio de trabajo a la carpeta anterior

if original_wdir == False:
    original_wdir = os.getcwd()
    os.chdir('../')
    new_wdir = os.getcwd() 

else:
    os.chdir(original_wdir)
    os.chdir('../')
    new_wdir = os.getcwd() 

print('Change directory:', original_wdir, '>>>', new_wdir)
# os.chdir(infolder)

Change directory: d:\Users\Andriu\Main\DOCUMENTOS\3. MACHINE LEARNING\PYTHON_PROYECTS\mlp_gifmanager\.notebooks >>> d:\Users\Andriu\Main\DOCUMENTOS\3. MACHINE LEARNING\PYTHON_PROYECTS\mlp_gifmanager


CARGAR SOURCES

In [26]:
input_files = [x for x in os.listdir(infolder) if x.endswith('.webp')]

input_names = [x.split('.webp')[0] for x in input_files]
input_paths = [infolder +  x for x in input_files]

gif_files = []

if verbose:
    for i in tqdm.tqdm(range(len(input_files)), ncols=100, desc='Loading gif files'):
        gif_files.append({'name':input_names[i], 'file':input_files[i], 'input_path':input_paths[i]})
else:
    for i in range(len(input_files)):
        gif_files.append({'name':input_names[i], 'file':input_files[i], 'path':input_paths[i]})


Loading gif files: 100%|██████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


EXTRAER FRAMES WEBP

In [15]:
def extract_frames(gif_metadata, output_folder):
    # Abre el archivo GIF
    gif = Image.open(gif_metadata['input_path'])

    fmax = Image.open(gif_files[0]['input_path']).n_frames
    lenght = len(str(fmax))

    # Itera sobre cada frame del GIF
    for i in range(gif.n_frames):
        # Selecciona el frame actual
        gif.seek(i)
        # Crea un nombre de archivo único para el frame actual
        frame_filename = output_folder + 'frames/' +  gif_metadata['name'] + '_' + str(i+1).zfill(lenght) + '.png'
        # Guarda el frame actual como un archivo PNG
        gif.convert("RGBA").save(frame_filename)

In [16]:
os.mkdir(outfolder + 'frames/')

In [17]:
extract_frames(gif_files[0], outfolder)

CARGAR FRAMES

In [29]:
def transform_webp_pil(gif_metadata):
    im = Image.open(gif_metadata['input_path'])
    im.info.pop('background', None)
    im = im.convert('RGB').convert('P', palette=Image.ADAPTIVE, colors=255)
    im.save(outfolder + 'result.gif', 'gif', save_all=True, lossless=True, quality=100, method=6)

In [30]:
transform_webp_pil(gif_files[0])

In [63]:
def transform_webp_pil(gif_metadata):
    im = Image.open(gif_metadata['input_path'])
    
    alpha = im.split()[-1]
    im.putalpha(alpha)
    im.info.pop('background', None)
    
    mask = Image.eval(alpha, lambda a: 255 if a <= 128 else 0)
    im.paste(255, mask)
    im.info['transparency'] = 255
    
    output_path = outfolder + gif_metadata['name'] + '.gif'
    im.save(output_path, 'gif', save_all=True)

In [64]:
transform_webp_pil(gif_files[0])

In [69]:
webp = imageio.imread('gif_builder/input/webp/demo.webp')
gif = imageio.imread('gif_builder/output/demo1.gif') 

In [71]:
from moviepy.editor import ImageSequenceClip

In [81]:
webp = ImageSequenceClip('gif_builder/input/webp/', fps=24)
# webp.write_gif('gif_builder/output/demo1.gif', fps=24)
webp.write_videofile("gif_builder/output/video.mp4", bitrate="3000k", fps=24)

Moviepy - Building video gif_builder/output/video.mp4.
Moviepy - Writing video gif_builder/output/video.mp4



Moviepy - Done !
Moviepy - video ready gif_builder/output/video.mp4


In [ ]:
    alpha = img.split()[-1]
    img.putalpha(alpha)
    img.info.pop('background', None)

    # the below convert method seems to output a single frame rather than the entire gif
    # perhaps, there is a simple method that converts every gif frame?
    # img = img.convert('RGB').convert('P', palette=Image.ADAPTIVE, colors=255)

    mask = Image.eval(alpha, lambda a: 255 if a <= 128 else 0)
    img.paste(255, mask)
    img.info['transparency'] = 255

    img.save(f'{fname}.gif', 'gif', save_all=True)
    return img

In [51]:
def transform_webp(gif_metadata):

    # reader = imageio.get_reader(gif_metadata['input_path'])
    # images = list(reader)

    # imageio.mimsave(gif_metadata['output_path'], images, format = 'GIF')

    clip = VideoFileClip(gif_metadata['input_path'])
    clip.write_gif(outfolder + gif_metadata['name'] + '.gif')

In [54]:
transform_webp(gif_files[0])

OSError: MoviePy error: failed to read the duration of file gif_builder/input/webp/demo.webp.
Here are the file infos returned by ffmpeg:

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 9.2.1 (GCC) 20200122
  configuration: --enable-gpl --enable-version3 --enable-sdl2 --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libdav1d --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libaom --enable-libmfx --enable-amf --enable-ffnvcodec --enable-cuvid --en